# Deconvolution of the LArSoft opdet simulation.

### This notebook is used to analyse wvfs of the deconvolution module in larsoft. The output of the workflow is a collection of hits. The hits are then converted into a format that can be compared against the true photon information.

1. The wvfs are loaded. For a full comparison, the current "IDEAL" template is compared agains the "RAW" wvf generated from the arapuca temnplate and the "DEC" wvf generated from the deconvolution module (using both "GAUSS" and "WIENER" filters).

In [ ]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *
# Load data according to type: RAW (digitized), DEC (deconvolved)
max_ev = 20
%time data0 = load_root_new("RAW","../data/DAPHNE_FBK/opdetraw_ideal_hist.root",    ["opdigi","opdigiana"],np.loadtxt("../template/MILANO/FBK/DAPHNE2_FBK_2022.txt"),max_ev=max_ev,label="IDEAL")
%time data1 = load_root_new("RAW","../data/DAPHNE_FBK/opdetraw_template_hist.root", ["opdigi","opdigiana"],np.loadtxt("../template/MILANO/FBK/DAPHNE2_FBK_2022.txt"),max_ev=max_ev,label="RAW")
%time data2 = load_root_new("DEC","../data/DAPHNE_FBK/deconv_gauss_hist.root",      ["opdigi","opdecoana"],np.loadtxt("../template/MILANO/FBK/DAPHNE2_FBK_2022.txt"),max_ev=max_ev,label="GAUSS")
%time data3 = load_root_new("DEC","../data/DAPHNE_FBK/deconv_wiener_hist.root",     ["opdigi","opdecoana"],np.loadtxt("../template/MILANO/FBK/DAPHNE2_FBK_2022.txt"),max_ev=max_ev,label="WIENER")

# Select your favorite color map
color_map={data0["LABEL"]:"violet", 
           data1["LABEL"]:"#3366CC", 
           data2["LABEL"]:"#66AA00", 
           data3["LABEL"]:"#FF9900"
           }
# sandybrown,seagreen,seashell,sienna,silver,skyblue,slateblue,slategray,slategrey,snow,springgreen,steelblue,tan,teal,thistle,tomato,turquoise,violet,wheat,white,whitesmoke,yellow,yellowgreen

2. The ophit data is loaded separetly. The ophit data is then converted into a format that can be compared against the true photons.

In [ ]:
# Load ophit data for each type
branches = ["Amplitude","EventID","OpChannel","PE","PeakTime","Width"]
%time raw_reco0 = ophit_data("../data/DAPHNE_FBK/ophitspe_ideal_hist.root", scaling=1,label="IDEAL",   branches=branches,as_df=False)
%time raw_reco1 = ophit_data("../data/DAPHNE_FBK/ophitspe_raw_hist.root",   scaling=1,label="RAW",     branches=branches,as_df=False)
%time raw_reco2 = ophit_data("../data/DAPHNE_FBK/ophitspe_gauss_hist.root", scaling=300,label="GAUSS", branches=branches,as_df=False)
%time raw_reco3 = ophit_data("../data/DAPHNE_FBK/ophitspe_wiener_hist.root",scaling=100,label="WIENER",branches=branches,as_df=False)

%time reco0 = order_ophit_data(raw_reco0,max_ev,data0["RECO"]["CH"])
%time reco1 = order_ophit_data(raw_reco1,max_ev,data1["RECO"]["CH"])
%time reco2 = order_ophit_data(raw_reco2,max_ev,data2["RECO"]["CH"])
%time reco3 = order_ophit_data(raw_reco3,max_ev,data3["RECO"]["CH"])

3. Finally, the hits are compared against the true photons and exported to the same format as the wvf data.

In [ ]:
# Combine data and ophit data
%time ophit0 = combine_data_ophit(data0,reco0)
%time ophit1 = combine_data_ophit(data1,reco1)
%time ophit2 = combine_data_ophit(data2,reco2)
%time ophit3 = combine_data_ophit(data3,reco3)

### Visualize individual wvfs according to event, channel and wvf number.

In [ ]:
# Choose between a random channel number or select yourself
# ev = 1; ch = 389; wf = 0
ev = np.random.choice(data1["RECO"]["EV"])
ch = np.random.choice(data1["RECO"]["CH"][data1["RECO"]["EV"] == ev])
wf = np.random.choice(data1["RECO"]["#WVF"][(data1["RECO"]["EV"] == ev)*(data1["RECO"]["CH"] == ch)])

# Find the index of the channel number in the data
try:
    num0 = np.where((data0["RECO"]["EV"] == ev)*(data0["RECO"]["CH"] == ch)*(data0["RECO"]["#WVF"] == wf))[0][0]
except:
    num0 = 0
    print("No ideal data for %s %s %s"%(ev,ch,wf))
    
num1 = np.where((data1["RECO"]["EV"] == ev)*(data1["RECO"]["CH"] == ch)*(data1["RECO"]["#WVF"] == wf))[0][0]

# Genarte figure with subplots
fig = make_subplots(rows=2, cols=2,subplot_titles=('', ''))

# Add waveform traces to the figure
fig.add_trace(go.Scatter(line=dict(color=color_map[data0["LABEL"]]),name="%s #PE: %.2f"%(data0["LABEL"],data0["RECO"]["PE"][num0]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data0["RECO"]["WVF_IX"][num0],y=data0["RECO"]["WVF"][num0]-data0["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data1["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data1["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data1["RECO"]["WVF_IX"][num1],y=data1["RECO"]["WVF"][num1]-data1["PEDESTAL"]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data2["LABEL"]]),name="%s #PE: %.2f"%(data2["LABEL"],data2["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data2["RECO"]["WVF_IX"][num1],y=data2["RECO"]["WVF"][num1]),col=1,row=2)
fig.add_trace(go.Scatter(line=dict(color=color_map[data3["LABEL"]]),name="%s #PE: %.2f"%(data3["LABEL"],data3["RECO"]["PE"][num1]),x=1e6*16e-9*(np.linspace(0,1000,1001))+data3["RECO"]["WVF_IX"][num1],y=data3["RECO"]["WVF"][num1]),col=2,row=2)

# Add vertical lines according to the reconstructed T0
fig.add_vline(x=data0["RECO"]["T0"][num0], line_width=2, line_dash="dash", line_color="gray",col=1,row=1)
fig.add_vline(x=data1["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=2,row=1)
fig.add_vline(x=data2["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=1,row=2)
fig.add_vline(x=data3["RECO"]["T0"][num1], line_width=2, line_dash="dash", line_color="gray",col=2,row=2)

# Add true PE times to the figure
# if len(data0["TRUE"]["PETIMES"][num0]) > 100:
# fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data0["TRUE"]["PETIMES"][num0])),x=np.asarray(data0["TRUE"]["PETIMES"][num0])),col=1,row=1)
# fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1])),col=2,row=1)
# else:
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data0["TRUE"]["PETIMES"][num0])),x=np.asarray(data0["TRUE"]["PETIMES"][num0]),y=np.zeros(len(data0["TRUE"]["PETIMES"][num0]))),col=1,row=1)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=2,row=1)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=1,row=2)
fig.add_trace(go.Scatter(marker_symbol="triangle-up",mode="markers",line=dict(color="black"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num1])),x=np.asarray(data1["TRUE"]["PETIMES"][num1]),y=np.zeros(len(data1["TRUE"]["PETIMES"][num1]))),col=2,row=2)

# Add ophits to the figure
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit0["PE"][num0]),marker=dict(color="#DC3912"),mode="markers",x=ophit0["TIMES"][num0],y=ophit0["AMP"][num0]/reco0["SCALING"],error_x=dict(arrayminus=np.zeros(len(ophit0["WIDTH"][num0])),array=ophit0["WIDTH"][num0])),col=1,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit1["PE"][num1]),marker=dict(color="#DC3912"),mode="markers",x=ophit1["TIMES"][num1],y=ophit1["AMP"][num1]/reco1["SCALING"],error_x=dict(arrayminus=np.zeros(len(ophit1["WIDTH"][num1])),array=ophit1["WIDTH"][num1])),col=2,row=1)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit2["PE"][num1]),marker=dict(color="#DC3912"),mode="markers",x=ophit2["TIMES"][num1],y=ophit2["AMP"][num1]/reco2["SCALING"],error_x=dict(arrayminus=np.zeros(len(ophit2["WIDTH"][num1])),array=ophit2["WIDTH"][num1])),col=1,row=2)
fig.add_trace(go.Scatter(name="OPHITFINDER #PE: %.2f"%(ophit3["PE"][num1]),marker=dict(color="#DC3912"),mode="markers",x=ophit3["TIMES"][num1],y=ophit3["AMP"][num1]/reco3["SCALING"],error_x=dict(arrayminus=np.zeros(len(ophit3["WIDTH"][num1])),array=ophit3["WIDTH"][num1])),col=2,row=2)

# Update the layout
fig.update_layout(title="Comparison for ev %i, ch %i and wvf %i"%(ev,ch,wf),xaxis_title="Time in [&mu;s]",yaxis_title="Amp. in [ADC]",xaxis2_title="Time in [&mu;s]",yaxis2_title="Amp. in [ADC]",xaxis3_title="Time in [&mu;s]",yaxis3_title="Amp. in [a.u.]",xaxis4_title="Time in [&mu;s]",yaxis4_title="Amp. in [a.u.]")
fig.update_layout(autosize=True,height=600,font=dict(size=16))
# fig.update_layout(template="presentation")
# fig.update_yaxes(type="log")
fig.show()

### Calculate further variables and formar data to generate a dataframe.

In [ ]:
raw_corr = 1.6284
corrt_raw_pe = raw_corr*np.asarray(ophit1["PE"])
with np.errstate(divide='ignore', invalid='ignore'):
    pe_error_0 = (np.asarray(data0["RECO"]["PE"])-np.asarray(data0["TRUE"]["PE"]))/np.asarray(data0["TRUE"]["PE"])
    pe_error_1 = (np.asarray(data1["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    pe_error_2 = (np.asarray(data2["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    pe_error_3 = (np.asarray(data3["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_0 = (np.asarray(ophit0["PE"])-np.asarray(data0["TRUE"]["PE"]))/np.asarray(data0["TRUE"]["PE"])
    ophitpe_error_1 = (np.asarray(corrt_raw_pe)-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_2 = (np.asarray(ophit2["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
    ophitpe_error_3 = (np.asarray(ophit3["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

    ophit_num_0 = []
    ophit_num_1 = []
    ophit_num_2 = []
    ophit_num_3 = []
    for i in range(len(pe_error_0)):
        ophit_num_0.append(ophit0["TIMES"][i].size)
    for i in range(len(pe_error_1)):
        ophit_num_1.append(ophit1["TIMES"][i].size)
    for i in range(len(pe_error_2)):
        ophit_num_2.append(ophit2["TIMES"][i].size)
    for i in range(len(pe_error_3)):
        ophit_num_3.append(ophit3["TIMES"][i].size)

    t0_reco_0 = np.asarray(data0["RECO"]["T0"])-np.asarray(data0["TRUE"]["T0"])
    t0_reco_1 = np.asarray(data1["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
    t0_reco_2 = np.asarray(data2["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])
    t0_reco_3 = np.asarray(data3["RECO"]["T0"])-np.asarray(data1["TRUE"]["T0"])

    ev            = np.concatenate([data0["RECO"]["EV"],  data1["RECO"]["EV"],  data2["RECO"]["EV"],  data3["RECO"]["EV"]])
    ch            = np.concatenate([data0["RECO"]["CH"],  data1["RECO"]["CH"],  data2["RECO"]["CH"],  data3["RECO"]["CH"]])
    wvf           = np.concatenate([data0["RECO"]["#WVF"],data1["RECO"]["#WVF"],data2["RECO"]["#WVF"],data3["RECO"]["#WVF"]])
    amp           = np.concatenate([data0["RECO"]["AMP"], data1["RECO"]["AMP"], data2["RECO"]["AMP"], data3["RECO"]["AMP"]])
    pe_reco       = np.concatenate([data0["RECO"]["PE"],  data1["RECO"]["PE"],  data2["RECO"]["PE"],  data3["RECO"]["PE"]])
    pe_true       = np.concatenate([data0["TRUE"]["PE"],  data1["TRUE"]["PE"],  data1["TRUE"]["PE"],  data1["TRUE"]["PE"]])
    pe_ophit      = np.concatenate([ophit0["PE"],         corrt_raw_pe,         ophit2["PE"],         ophit3["PE"]])
    ophit_num     = np.concatenate([ophit_num_0,          ophit_num_1,          ophit_num_2,          ophit_num_3])
    ophitpe_error = np.concatenate([ophitpe_error_0,      ophitpe_error_1,      ophitpe_error_2,      ophitpe_error_3])
    pe_error      = np.concatenate([pe_error_0,           pe_error_1,           pe_error_2,           pe_error_3])
    t0_reco       = np.concatenate([t0_reco_0,            t0_reco_1,            t0_reco_2,            t0_reco_3])
    filter_label  = np.concatenate([[data0["LABEL"]]*len(pe_error_0),[data1["LABEL"]]*len(pe_error_1),[data2["LABEL"]]*len(pe_error_2),[data3["LABEL"]]*len(pe_error_3)])

    df = pd.DataFrame({ "EV":ev,
                        "CH":ch,
                        "WVF":wvf,
                        "FILTER":filter_label,
                        "ERROR PE":pe_error,
                        "ERROR OPHIT PE":ophitpe_error,
                        "TRUE PE":pe_true,
                        "RECO PE":pe_reco, 
                        "OPHIT PE":pe_ophit,
                        "OPHIT NUM":ophit_num,
                        "AMP":amp,
                        "RECO T0":t0_reco})

### Gauss fit to histogram. This can be used to calculate the SPE resolution.

In [ ]:
min_pe = 5; max_pe = 15
this_df = df[(df["TRUE PE"] > min_pe)*(df["TRUE PE"] < max_pe)*(df["OPHIT PE"] > 0)]
variable = "ERROR OPHIT PE"
xlim=(1,99)
data = []
acc = 50
fig,data = gauss_fit_distribution(this_df,this_bin=5,variable=variable,color="FILTER",color_map=color_map,output_data=data,xlim=xlim,acc=20,terminal=True)
fig.update_layout(bargap = 0,)
fig.update_layout(title="RECO PE ERROR (%i - %i TRUE PE)"%(min_pe,max_pe),xaxis_title="PE ERROR [(RECO PE - TRUE PE)/TRUE PE]",yaxis_title="NORM",height=800)
fig.show()

In [ ]:
pe_min = 2; pe_max = 100
variable = "ERROR OPHIT PE"
pe_bins = np.arange(pe_min,pe_max,2)
pe_bin_centers = (pe_bins[1:] + pe_bins[:-1])/2
dict_list = []
for idx,pe_bin in enumerate(pe_bin_centers):
    for jdx,filter_label in enumerate(["IDEAL","RAW","GAUSS","WIENER"]):
        this_df = df[(df["TRUE PE"] > pe_bins[idx])*(df["TRUE PE"] < pe_bins[idx+1])*(df["OPHIT PE"] > 0)*(df["FILTER"] == filter_label)]
        this_dict = {"PE":pe_bin,
                    "FILTER":filter_label,
                    "MEAN '%'":100*np.mean(this_df[variable]),
                    "STD '%'":100*np.std(this_df[variable])/np.sqrt(pe_bin)}
        dict_list.append(this_dict)
std_df = pd.DataFrame(dict_list)

fig = make_subplots(rows=1, cols=2,subplot_titles=("MEAN RESIDUAL (%i-%i PE)"%(pe_min,pe_max),"STD RESIDUAL (%i-%i PE)"%(pe_min,pe_max)))
for i in range(4):
    fig.add_trace(px.line(data_frame=std_df,
                x="PE",
                y="MEAN '%'",title="MEAN RESIDUAL (%i-%i PE)"%(pe_min,pe_max), 
                # error_y="STD '%'",
                color="FILTER",
                color_discrete_map=color_map,
                ).data[i],row=1,col=1)
    fig.add_trace(px.line(data_frame=std_df,
                x="PE",
                y="STD '%'",title="STD RESIDUAL (%i-%i PE)"%(pe_min,pe_max),log_y=True,
                color="FILTER",
                color_discrete_map=color_map,
                ).data[i],row=1,col=2)

fig = format_coustom_plotly(fig,tickformat=(",.2s",",.2s"),figsize=(None,600),fontsize=18)
fig.update_layout(xaxis1_title="TRUE PE",yaxis1_title="MEAN RESIDUAL (%)",height=600)
fig.update_layout(xaxis2_title="TRUE PE",yaxis2_title="STD RESIDUAL (%)",height=600)
# fig.update_yaxes(tickformat=",.0f") # use thousand comma; round to whole number
fig.show()


In [ ]:
this_df = df[(df["TRUE PE"] > 0) & (df["TRUE PE"] < 5) & (df["OPHIT NUM"] < 5) & (df["OPHIT NUM"] != 0)]
fig = px.histogram(data_frame=this_df,
                x="OPHIT PE",
                facet_col="FILTER",
                color="FILTER",
                color_discrete_map=color_map,
                histnorm='percent',
                nbins=20,
                )
fig = format_coustom_plotly(fig,tickformat=(",.0s",".0s"),log=(False,True),figsize=(None,600),fontsize=20,facet_titles=["IDEAL","RAW","GAUSS","WIENER"])
fig.update_yaxes(showgrid=True) # use thousand comma; round to whole number
fig.show()

### This shows a regular histogram.

In [ ]:
t0_df = df[(df['TRUE PE'] > 20)*(df['TRUE PE'] < 200)]
fig = px.histogram(data_frame=t0_df,
                    x="RECO T0",
                    color='FILTER',
                    color_discrete_map=color_map,
                    barmode="overlay",
                    # marginal='box',
                    nbins=5000,
                    histnorm="percent",
                    )
for i,ind in enumerate(t0_df["FILTER"].unique()):
    this_df = t0_df[t0_df["FILTER"] == ind]
    mode = this_df["RECO T0"].mode().values
    fig.add_vline(x=mode[0],line_width=3,line_dash="dash",line_color=color_map[ind],annotation={"yshift":-i*20,"text":"MODE: %.3f"%mode[0],"showarrow":False})

fig = format_coustom_plotly(fig,tickformat=(",.2f",".2f"),figsize=(None,600),fontsize=18,ranges=([-0.16,0.48],None))
fig.update_layout(bargap=0,title="T0 RECOVERY",xaxis_title="Peak Time - True PE Time [us]",autosize=True,height=800)
fig.show()

### Finally a scintillation fit of the deconvolved wvfs is shown.

In [ ]:
min_wvf_length = 1000
ave_wvfs = []
for data in [data2,data3]:

    # Select only the waveforms with a minimum length
    data["RECO"]["SHORT_WVF"] = []
    for wvf in data["RECO"]["WVF"]:
        if len(wvf) >= min_wvf_length:
            data["RECO"]["SHORT_WVF"].append(wvf[:min_wvf_length])
    print("Number of waveforms with length > %i: %i"%(min_wvf_length,len(data["RECO"]["SHORT_WVF"])))
    # Prepare the averaged deconvolved waveforms for the scintillation profile fit
    ave_wvf = np.mean(np.asarray(data["RECO"]["SHORT_WVF"]),axis=0)
    ave_wvf = ave_wvf/np.max(ave_wvf)
    ave_wvf = ave_wvf[np.argmax(ave_wvf):np.argmax(ave_wvf)+500]

    ave_wvfs.append(ave_wvf)
    print(len(ave_wvf))
    # Fit the scintillation profile of the averaged deconvolved waveforms
    initial = [1.8e2,1.7e-1,6,1.6e3]
    labels = ["CONSTANT","AMPLITUDE","TAU FAST","TAU SLOW"]
    try:
        popt, pcov = curve_fit(scint_profile,16*np.arange(len(ave_wvf)),ave_wvf,p0=initial)
        perr = np.sqrt(np.diag(pcov))
    except RuntimeError:
        print("ERROR: Fit failed for " + data["LABEL"])
        popt = initial
        perr = np.zeros(len(initial))
    # Print the fit results
    ave_wvfs.append(scint_profile(16*np.arange(len(ave_wvf)),*popt))
    print("\n----------- FIT VALUES " + data["LABEL"]+ " ------------")
    for i in range(len(initial)):
        print("%s:\t%.2E\t%.2E"%(labels[i], popt[i], perr[i]))
    print("-----------------------------------------")
    print(len(ave_wvf))

In [ ]:
# Generate the dataframe for the plot
wvfs  = np.concatenate([ave_wvfs[0],ave_wvfs[1],ave_wvfs[2],ave_wvfs[3]])
wvfs_x  = np.concatenate([16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf)),16*np.arange(len(ave_wvf))])
fit_label  = np.concatenate([["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf),["WVF"]*len(ave_wvf),["FIT"]*len(ave_wvf)])
filter_label  = np.concatenate([[data2["LABEL"]]*len(ave_wvf),[data2["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf),[data3["LABEL"]]*len(ave_wvf)])

fit_df = pd.DataFrame({"WVF":wvfs,"TIME in [ns]":wvfs_x,"FILTER":filter_label,"FIT":fit_label})

# Plot the results
fig = px.line(data_frame=fit_df,x="TIME in [ns]",y="WVF",color="FIT",facet_col="FILTER",log_y=True,color_discrete_map=color_map)
fig = format_coustom_plotly(fig,tickformat=(",.2s",".2s"),figsize=(None,600),fontsize=18,ranges=(None,None),facet_titles=["GAUSS","WIENER"])
fig.show()